In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, cross_val_predict, RandomizedSearchCV
from sklearn.metrics import root_mean_squared_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [21]:
data = pd.read_csv('AirQualityUCI.csv')
data.head()

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,Unnamed: 15,Unnamed: 16
0,3/10/2004,18:00:00,2.6,1360.0,150.0,11.9,1046.0,166.0,1056.0,113.0,1692.0,1268.0,13.6,48.9,0.7578,NaN,NaN
1,3/10/2004,19:00:00,2.0,1292.0,112.0,9.4,955.0,103.0,1174.0,92.0,1559.0,972.0,13.3,47.7,0.7255,NaN,NaN
2,3/10/2004,20:00:00,2.2,1402.0,88.0,9.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,11.9,54.0,0.7502,NaN,NaN
3,3/10/2004,21:00:00,2.2,1376.0,80.0,9.2,948.0,172.0,1092.0,122.0,1584.0,1203.0,11.0,60.0,0.7867,NaN,NaN
4,3/10/2004,22:00:00,1.6,1272.0,51.0,6.5,836.0,131.0,1205.0,116.0,1490.0,1110.0,11.2,59.6,0.7888,NaN,NaN


In [22]:
# Rename the columns to be more descriptive
data.rename(columns={
    'Date': 'Measurement_Date',
    'Time': 'Measurement_Time',
    'CO(GT)': 'Carbon_Monoxide_Concentration',
    'PT08.S1(CO)': 'CO_Sensor_Measurement',
    'NMHC(GT)': 'Non_Methane_Hydrocarbons_Concentration',
    'C6H6(GT)': 'Benzene_Concentration',
    'PT08.S2(NMHC)': 'NMHC_Sensor_Measurement',
    'NOx(GT)': 'Nitrogen_Oxides_Concentration',
    'PT08.S3(NOx)': 'NOx_Sensor_Measurement',
    'NO2(GT)': 'Nitrogen_Dioxide_Concentration',
    'T': 'Temperature_Celsius',
    'RH': 'Relative_Humidity',
    'AH': 'Absolute_Humidity'
}, inplace=True)

In [23]:
# Visualize the dataset
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9471 entries, 0 to 9470
Data columns (total 17 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Measurement_Date                        9357 non-null   object 
 1   Measurement_Time                        9357 non-null   object 
 2   Carbon_Monoxide_Concentration           9357 non-null   float64
 3   CO_Sensor_Measurement                   9357 non-null   float64
 4   Non_Methane_Hydrocarbons_Concentration  9357 non-null   float64
 5   Benzene_Concentration                   9357 non-null   float64
 6   NMHC_Sensor_Measurement                 9357 non-null   float64
 7   Nitrogen_Oxides_Concentration           9357 non-null   float64
 8   NOx_Sensor_Measurement                  9357 non-null   float64
 9   Nitrogen_Dioxide_Concentration          9357 non-null   float64
 10  PT08.S4(NO2)                            9357 non-null   floa

In [24]:
# Remove weird last two columns
data.drop(data.columns[15:17], axis=1, inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9471 entries, 0 to 9470
Data columns (total 15 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Measurement_Date                        9357 non-null   object 
 1   Measurement_Time                        9357 non-null   object 
 2   Carbon_Monoxide_Concentration           9357 non-null   float64
 3   CO_Sensor_Measurement                   9357 non-null   float64
 4   Non_Methane_Hydrocarbons_Concentration  9357 non-null   float64
 5   Benzene_Concentration                   9357 non-null   float64
 6   NMHC_Sensor_Measurement                 9357 non-null   float64
 7   Nitrogen_Oxides_Concentration           9357 non-null   float64
 8   NOx_Sensor_Measurement                  9357 non-null   float64
 9   Nitrogen_Dioxide_Concentration          9357 non-null   float64
 10  PT08.S4(NO2)                            9357 non-null   floa

In [25]:
# List columns with missing data
missing_data = data.isna().any()
columns_with_missing_data = missing_data[missing_data].index.tolist()
print(columns_with_missing_data)

['Measurement_Date', 'Measurement_Time', 'Carbon_Monoxide_Concentration', 'CO_Sensor_Measurement', 'Non_Methane_Hydrocarbons_Concentration', 'Benzene_Concentration', 'NMHC_Sensor_Measurement', 'Nitrogen_Oxides_Concentration', 'NOx_Sensor_Measurement', 'Nitrogen_Dioxide_Concentration', 'PT08.S4(NO2)', 'PT08.S5(O3)', 'Temperature_Celsius', 'Relative_Humidity', 'Absolute_Humidity']


In [26]:
# After visual inspection, it looks like there are only 9358 entries in the csv file. Not sure why data.info() is registering 9471 
# data entries? Because of this, I will only "keep" the 9358 rows.
data = data[:9357]
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9357 entries, 0 to 9356
Data columns (total 15 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Measurement_Date                        9357 non-null   object 
 1   Measurement_Time                        9357 non-null   object 
 2   Carbon_Monoxide_Concentration           9357 non-null   float64
 3   CO_Sensor_Measurement                   9357 non-null   float64
 4   Non_Methane_Hydrocarbons_Concentration  9357 non-null   float64
 5   Benzene_Concentration                   9357 non-null   float64
 6   NMHC_Sensor_Measurement                 9357 non-null   float64
 7   Nitrogen_Oxides_Concentration           9357 non-null   float64
 8   NOx_Sensor_Measurement                  9357 non-null   float64
 9   Nitrogen_Dioxide_Concentration          9357 non-null   float64
 10  PT08.S4(NO2)                            9357 non-null   floa

In [27]:
# Isolate the numerical data columns
numerical_data = data.iloc[:, 2:14]

# The dataset notes that some measurements may be recorded as -200, indicating missing or invalid data points. I will remove 
# any -200 values and replace them with the median value in the column.
numerical_data.replace(-200, pd.NA, inplace=True)

# Replace NaN values with the median of each column
numerical_data.fillna(numerical_data.mean(), inplace=True)

/tmp/ipykernel_5683/3925680732.py:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  numerical_data.fillna(numerical_data.mean(), inplace=True)


In [28]:
# Calculate the correlation of the target (Nitrogen Dioxide Concentration) with the other features
correlations = numerical_data.corrwith(numerical_data['Nitrogen_Dioxide_Concentration'])
print(correlations)

Carbon_Monoxide_Concentration             0.646349
CO_Sensor_Measurement                     0.562843
Non_Methane_Hydrocarbons_Concentration    0.156510
Benzene_Concentration                     0.535919
NMHC_Sensor_Measurement                   0.561180
Nitrogen_Oxides_Concentration             0.763074
NOx_Sensor_Measurement                   -0.569479
Nitrogen_Dioxide_Concentration            1.000000
PT08.S4(NO2)                              0.140655
PT08.S5(O3)                               0.629443
Temperature_Celsius                      -0.165466
Relative_Humidity                        -0.080569
dtype: float64


In [15]:
# Showing strong correlation with the Carbon_Monoxide_Concentration, Nitrogen_Oxides_Concentration features

In [29]:
#Split the dataset into a train and test set using Nitrogen_Dioxide_Concentration as the target label
train_set, test_set = train_test_split(numerical_data, test_size=0.2, random_state=42)

#Begin preparation of data for ML algorithms with fresh copy of training data
train_copy = train_set.copy()

#Separate the predictors and the label
train_data = train_copy.drop('Nitrogen_Dioxide_Concentration', axis=1)
train_labels = train_copy['Nitrogen_Dioxide_Concentration'].copy()

In [30]:
#Create pipelines for numerical attributes
#Imputer with median strategy for any missing values and then a standard scaler is used on the numerical data
num_pipeline = make_pipeline(SimpleImputer(strategy="mean"), StandardScaler())

# Select columns with numerical attributes
preprocessing = make_column_transformer(
    (num_pipeline, make_column_selector(dtype_include=np.number)),
)

# Create the complete pipeline
pipeline = Pipeline(steps=[
    ('preprocessing', preprocessing),  # Step for preprocessing
    ('randomforestregressor', RandomForestRegressor(random_state=42))  # RandomForestRegressor step
])

In [31]:
#Create pipeline with preprocessing and a RandomForestClassifier with 10 cross validation folds
forest_regressor = make_pipeline(preprocessing, RandomForestRegressor(random_state=42))

# Define the hyperparameter grid to search
param_grid = {
    'randomforestregressor__n_estimators': [50, 100, 150, 200, 300],
    'randomforestregressor__max_depth': [5, 10, 20, 30],
    'randomforestregressor__min_samples_split': list(range(2, 21)),  # Min samples to split a node
    'randomforestregressor__min_samples_leaf': list(range(1, 11)),  # Min samples at a leaf node
}
# Set up the randomized search with 10 cross-validation folds, 100 combinations to try, and the negative mean squared error.
random_search = RandomizedSearchCV(
    estimator=forest_regressor,
    param_distributions=param_grid,
    n_iter=100,
    scoring='neg_mean_squared_error',
    cv=10,
    verbose=0,
    random_state=42,
    n_jobs=-1
)
# Fit on the training data
random_search.fit(train_data, train_labels)

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/lib/python3/dist-packages/scipy

RandomizedSearchCV(cv=10,
                   estimator=Pipeline(steps=[('columntransformer',
                                              ColumnTransformer(transformers=[('pipeline',
                                                                               Pipeline(steps=[('simpleimputer',
                                                                                                SimpleImputer()),
                                                                                               ('standardscaler',
                                                                                                StandardScaler())]),
                                                                               <sklearn.compose._column_transformer.make_column_selector object at 0x7e0571cae740>)])),
                                             ('randomforestregressor',
                                              RandomForestRegressor(random_state=4...
                   n_iter=100, n_jobs=-1,
                   param_distributions={'randomforestregressor__max_depth': [5,
                                                                             10,
                                                                             20,
                                                                             30],
                                        'randomforestregressor__min_samples_leaf': [1,
                                                                                    2,
                                                                                    3,
                                                                                    4,
                                                                                    5,
                                                                                    6,
                                                                                    7,
                                                                                    8,
                                                                                    9,
                                                                                    10],
                                        'randomforestregressor__min_samples_split': [2,
                                                                                     3,
                                                                                     4,
                                                                                     5,
                                                                                     6,
                                                                                     7,
                                                                                     8,
                                                                                     9,
                                                                                     10,
                                                                                     11,
                                                                                     12,
                                                                                     13,
                                                                                     14,
                                                                                     15,
                                                                                     16,
                                                                                     17,
                                                                                     18,
                                                                                     19,
                                                                                     20],
                                        'randomforestregressor__n_estimators': [50,
                           

In [32]:
#Evaluate the model on the test set
test_data = test_set.drop("Nitrogen_Dioxide_Concentration", axis=1)
test_labels = test_set["Nitrogen_Dioxide_Concentration"].copy()

final_predictions = random_search.best_estimator_.predict(test_data)

In [33]:
# Calculate metrics
mse = mean_squared_error(test_labels, final_predictions)
rmse = root_mean_squared_error(test_labels, final_predictions)  # RMSE
r2 = r2_score(test_labels, final_predictions)

# Print results
print(f'Mean Squared Error: {mse}')
print(f'Root Mean Squared Error: {rmse}')
print(f'R² Score: {r2}')

Mean Squared Error: 206.5233261379872
Root Mean Squared Error: 14.37091946042379
R² Score: 0.8901709298058845


In [65]:
numerical_data["Nitrogen_Dioxide_Concentration"].max() - numerical_data["Nitrogen_Dioxide_Concentration"].min()

540.0

In [34]:
# RMSE suggests that the model's predictions deviate from actual values by about 14.37 µg/m³ on average. Relative
# to the range of 540, this is approximately 2.66%, which is pretty good.